# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse              Shipments  \
0  Jun 14 2023  2:14PM  264763        18            SEQ-9572395   
1  Jun 14 2023  2:12PM  264764        10              MSP222711   
2  Jun 14 2023  2:12PM  264764        10              MSP222714   
3  Jun 14 2023  2:12PM  264764        10              MSP222715   
4  Jun 14 2023  2:11PM  264762        10  MSP222712-Hold_Rx(MI)   
5  Jun 14 2023  2:11PM  264762        10  MSP222713-Hold_Rx(MI)   
6  Jun 14 2023  1:53PM  264761        22                  61423   
7  Jun 14 2023  1:33PM  264760        10              MSP222704   
8  Jun 14 2023  1:33PM  264760        10              MSP222705   
9  Jun 14 2023  1:33PM  264760        10              MSP222706   

            Customer ShipmentStatus  
0            Seqirus       Released  
1   Methapharm, Inc.       Released  
2   Methapharm, Inc.       Released  
3   Methapharm, Inc.       Released  
4   Methapharm, Inc.       Released  
5   Methapharm, Inc.       Released  
6  NBTY Global, Inc.      Executing  
7   Methapharm, Inc.       Released  
8   Methapharm, Inc.       Released  
9   Methapharm, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
27  264760       Released         10
28  264761      Executing          1
29  264762       Released          2
30  264763       Released          1
31  264764       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
264760                NaN        NaN      10.0
264761                NaN        1.0       NaN
264762                NaN        NaN       2.0
264763                NaN        NaN       1.0
264764                NaN        NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264703                0.0        0.0       1.0
264707               11.0        8.0       0.0
264713                0.0        0.0       2.0
264724                0.0        4.0       2.0
264726                0.0        0.0       3.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264703                  0          0         1
264707                 11          8         0
264713                  0          0         2
264724                  0          4         2
264726                  0          0         3

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               264703          0          0         1
1               264707         11          8         0
2               264713          0          0         2
3               264724          0          4         2
4               264726          0          0         3

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               264703                            1
1               264707        11         8         
2               264713                            2
3               264724                   4        2
4               264726                            3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                     Customer
0   Jun 14 2023  2:14PM  264763        18                      Seqirus
1   Jun 14 2023  2:12PM  264764        10             Methapharm, Inc.
4   Jun 14 2023  2:11PM  264762        10             Methapharm, Inc.
6   Jun 14 2023  1:53PM  264761        22            NBTY Global, Inc.
7   Jun 14 2023  1:33PM  264760        10             Methapharm, Inc.
17  Jun 14 2023  1:30PM  264759        10                  Emerginnova
20  Jun 14 2023  1:29PM  264758        10                    Hush Hush
25  Jun 14 2023  1:29PM  264757        20  Reliable 1 Laboratories LLC
49  Jun 14 2023  1:28PM  264756        10          Digital Brands, LLC
51  Jun 14 2023  1:27PM  264755        10                Alikai Health

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                     Customer  \
0  Jun 14 2023  2:14PM  264763        18                      Seqirus   
1  Jun 14 2023  2:12PM  264764        10             Methapharm, Inc.   
2  Jun 14 2023  2:11PM  264762        10             Methapharm, Inc.   
3  Jun 14 2023  1:53PM  264761        22            NBTY Global, Inc.   
4  Jun 14 2023  1:33PM  264760        10             Methapharm, Inc.   
5  Jun 14 2023  1:30PM  264759        10                  Emerginnova   
6  Jun 14 2023  1:29PM  264758        10                    Hush Hush   
7  Jun 14 2023  1:29PM  264757        20  Reliable 1 Laboratories LLC   
8  Jun 14 2023  1:28PM  264756        10          Digital Brands, LLC   
9  Jun 14 2023  1:27PM  264755        10                Alikai Health   

  Completed Executing Released  
0                            1  
1                            3  
2                            2  
3                   1           
4                           10  
5                            3  
6                            5  
7                           24  
8                            2  
9                            7

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                     Customer  \
0  Jun 14 2023  2:14PM  264763        18                      Seqirus   
1  Jun 14 2023  2:12PM  264764        10             Methapharm, Inc.   
2  Jun 14 2023  2:11PM  264762        10             Methapharm, Inc.   
3  Jun 14 2023  1:53PM  264761        22            NBTY Global, Inc.   
4  Jun 14 2023  1:33PM  264760        10             Methapharm, Inc.   
5  Jun 14 2023  1:30PM  264759        10                  Emerginnova   
6  Jun 14 2023  1:29PM  264758        10                    Hush Hush   
7  Jun 14 2023  1:29PM  264757        20  Reliable 1 Laboratories LLC   
8  Jun 14 2023  1:28PM  264756        10          Digital Brands, LLC   
9  Jun 14 2023  1:27PM  264755        10                Alikai Health   

  Released Executing Completed  
0        1                      
1        3                      
2        2                      
3                  1            
4       10                      
5        3                      
6        5                      
7       24                      
8        2                      
9        7

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer Released  \
0  Jun 14 2023  2:14PM  264763        18            Seqirus        1   
1  Jun 14 2023  2:12PM  264764        10   Methapharm, Inc.        3   
2  Jun 14 2023  2:11PM  264762        10   Methapharm, Inc.        2   
3  Jun 14 2023  1:53PM  264761        22  NBTY Global, Inc.            
4  Jun 14 2023  1:33PM  264760        10   Methapharm, Inc.       10   

  Executing Completed  
0                      
1                      
2                      
3         1            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Jun 14 2023  2:14PM  264763        18            Seqirus       1.0   
1  Jun 14 2023  2:12PM  264764        10   Methapharm, Inc.       3.0   
2  Jun 14 2023  2:11PM  264762        10   Methapharm, Inc.       2.0   
3  Jun 14 2023  1:53PM  264761        22  NBTY Global, Inc.       NaN   
4  Jun 14 2023  1:33PM  264760        10   Methapharm, Inc.      10.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        1.0        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Jun 14 2023  2:14PM  264763        18            Seqirus       1.0   
1  Jun 14 2023  2:12PM  264764        10   Methapharm, Inc.       3.0   
2  Jun 14 2023  2:11PM  264762        10   Methapharm, Inc.       2.0   
3  Jun 14 2023  1:53PM  264761        22  NBTY Global, Inc.       0.0   
4  Jun 14 2023  1:33PM  264760        10   Methapharm, Inc.      10.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        1.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3706512      56.0        8.0        0.0
15          529504      20.0       26.0        0.0
18          264763       1.0        0.0        0.0
19         1588384       4.0        9.0       11.0
20          794196      29.0        0.0        0.0
21          264746       1.0        0.0        0.0
22          264761       0.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3706512      56.0        8.0        0.0
1        15   529504      20.0       26.0        0.0
2        18   264763       1.0        0.0        0.0
3        19  1588384       4.0        9.0       11.0
4        20   794196      29.0        0.0        0.0
5        21   264746       1.0        0.0        0.0
6        22   264761       0.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      56.0        8.0        0.0
1        15      20.0       26.0        0.0
2        18       1.0        0.0        0.0
3        19       4.0        9.0       11.0
4        20      29.0        0.0        0.0
5        21       1.0        0.0        0.0
6        22       0.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   56.0
1        15  Released   20.0
2        18  Released    1.0
3        19  Released    4.0
4        20  Released   29.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   18    19    20   21   22
Status                                          
Completed   0.0   0.0  0.0  11.0   0.0  0.0  0.0
Executing   8.0  26.0  0.0   9.0   0.0  0.0  1.0
Released   56.0  20.0  1.0   4.0  29.0  1.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   18    19    20   21   22
0          Completed   0.0   0.0  0.0  11.0   0.0  0.0  0.0
1          Executing   8.0  26.0  0.0   9.0   0.0  0.0  1.0
2           Released  56.0  20.0  1.0   4.0  29.0  1.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   18    19    20   21   22
0  Completed   0.0   0.0  0.0  11.0   0.0  0.0  0.0
1  Executing   8.0  26.0  0.0   9.0   0.0  0.0  1.0
2   Released  56.0  20.0  1.0   4.0  29.0  1.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()